### Import Libs

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sqlite3
import ast
import re

### Df Params

In [5]:
get_email = True
af_by_name = True

### Getting df from db

In [6]:
columns = [f'Mise {i}' for i in range(1,6)] + [i for i in range(1,58)]
query_columns = 'Mises, AF'
if get_email:
    columns = ['Email'] + columns
    query_columns = query_columns + ', email'

df = pd.DataFrame(columns = columns)
AF_dict = pd.read_pickle('Dicionario.pkl')

conn = sqlite3.connect("banco_de_dados.db")
cursor = conn.cursor()
cmd = f"SELECT {query_columns} from questionnaire_s8 ORDER BY ID ASC"
cursor.execute(cmd)
query_result = cursor.fetchall()

for i in range(len(query_result)):
    disciplinas = ast.literal_eval(query_result[i][1])
    if af_by_name: disciplinas = list(map(lambda x: AF_dict[x],disciplinas))
    
    if len(disciplinas) < 57:
        missing = 57 - len(disciplinas)
        disciplinas += ['-' for i in range(missing)]
    
    if get_email:
        row = pd.Series([query_result[i][2]] + ast.literal_eval(query_result[i][0]) +  disciplinas, index=df.columns)
    else:
        row = pd.Series(ast.literal_eval(query_result[i][0]) +  disciplinas, index=df.columns)

    df = df.append(row, ignore_index=True)

### Creating Visualization

#### Organizing data

In [64]:
dict_AF = {}

for index in df.index:
    subjects = df.loc[index, [i+1 for i in range(57)]]

    for subject_index in range(1, subjects.shape[0]+1):
        subject = subjects.loc[subject_index]

        if subject in dict_AF:
            dict_AF[subject].append(subject_index)
        else: 
            dict_AF[subject] = [subject_index]

In [65]:
dict_mises = {}

for index in df.index:
    mises = df.loc[index, [f'Mise {i+1}' for i in range(5)]]

    for position in range(1, mises.shape[0]+1):
        mise = mises.loc[f'Mise {position}']

        if f'Mise {position}' in dict_mises:
            dict_mises[f'Mise {position}'].append(mise)
        else: 
            dict_mises[f'Mise {position}'] = [mise]


In [89]:
dict_AF = dict(reversed(sorted(dict_AF.items())))  # Afs in alphabetical order

#### Creating plot

In [ ]:
with PdfPages('histogramas.pdf') as pdf:
    for subject, classifications in dict_AF.items():
        if subject == '-':  # skip '-'
            continue

        plt.figure(figsize=(17,10))
        plt.hist(classifications, bins=range(1,59), align='left', rwidth=.8)

        loc, labels = plt.yticks()
        locx, labelsx = plt.xticks()
        plt.yticks(np.arange(0, max(loc), step=1))
        plt.grid(axis='y', alpha=0.5)
        plt.xticks(np.arange(1, 58, step=1))

        plt.title(subject, fontsize=20)
        plt.xlabel('Position', fontsize=20)
        plt.ylabel('Quantité de personnes', fontsize=20)
        pdf.savefig()

## Add new data 

In [267]:
sondage = pd.read_csv('S8 sondagem disciplinas (respostas) - Respostas ao formulário 1.csv')
sondage = sondage.drop(columns=sondage.columns[[0,1,3]])  # Remove useless columns

In [268]:
# Get name
sondage['name'] = sondage.apply(lambda row: row[sondage.columns[0]] if type(row[sondage.columns[0]]) == str else row[sondage.columns[1]] , axis=1)
sondage = sondage.drop(columns=sondage.columns[[0,1]])

In [269]:
# Assign one pick for a unique column
for i in range(5):
    df[f'Pick {i+1}'] = ['' for i in range(df.shape[0])]  # create new columns for df
    sondage[f'Pick {i+1}'] = sondage.iloc[:, 0].apply(lambda txt: re.split('[A-Z][0-9][0-9]?', txt)[i+1].strip(' ,'))
sondage = sondage.drop(columns=sondage.columns[0])

In [270]:
# try to add data automatically
pickColumns = [f'Pick {n+1}' for n in range(5)]
for row in sondage.iterrows():
    name = row[1]['name'].split(' ')[-1].lower()
    surname = row[1]['name'].split(' ')[0].lower()
    for i in range(df.shape[0]):
        if (name in df.iloc[i, 0]) and (surname in df.iloc[i, 0]):
            df.loc[i, pickColumns] = row[1][pickColumns]
            break

In [271]:
# add missing data by hand
from unicodedata import normalize
choices = sondage.loc[sondage.loc[:, 'name'].apply(lambda name: 'leo' in normalize('NFKD', name).encode('ASCII','ignore').decode('ASCII').lower())]
df.loc[df.iloc[:, 0].apply(lambda email: 'leandro' in email), pickColumns] = choices.loc[:, pickColumns].values

In [272]:
# order picks
for items in df.iterrows():
    df_index = items[0]
    items = items[1]

    subjects = items[pickColumns].values
    subjects = [normalize('NFKD', subject).encode('ASCII','ignore').decode('ASCII').lower() for subject in subjects]
    ordered_subjects = []

    for i in range(1,58):
        if items[i] == '-':
            break
        if normalize('NFKD', items[i]).encode('ASCII','ignore').decode('ASCII').lower() in subjects:
            ordered_subjects.append(items[i])

    while len(ordered_subjects) < 5:
        ordered_subjects.append('-')

    df.loc[df_index, pickColumns] = ordered_subjects

In [273]:
df

,Email,Mise 1,Mise 2,Mise 3,Mise 4,Mise 5,1,2,3,4,...,53,54,55,56,57,Pick 1,Pick 2,Pick 3,Pick 4,Pick 5
0,elias.martini@ecl21.ec-lyon.fr,30.0,27.0,20.0,13.0,10.0,Eoliennes,Finance d'entreprise,Programmation des interfaces graphiques en C++,Marketing,...,Mécanique des sols,Mécanique des structures minces : Plaques et C...,Théorie des probabilités et introduction aux p...,"Méthode des éléments finis, de la théorie à la...",Analyse fonctionnelle : théorie et applications,Eoliennes,Programmation des interfaces graphiques en C++,Conception d'un emballage responsable,Ingénierie et systèmes Haute Tension,An Introduction to Meteorology and Oceanography
1,enzo.da-rocha-paes-alves-barreto@ecl21.ec-lyon.fr,34.0,31.0,25.0,7.0,3.0,Finance d'entreprise,Finance de marché,Marketing,Entreprendre et innover,...,Propulseurs aéronautiques,Introduction aux vibrations aléatoires,Optique et Photonique pour l'Ingénieur,"Ordre, chaos, fractales",Analyse fonctionnelle : théorie et applications,Finance d'entreprise,Finance de marché,"Vivant, Information et Système",Conception d'un emballage responsable,Savoir choisir un matériau
2,luana.oliveira@ecl21.ec-lyon.fr,24.0,24.0,24.0,14.0,14.0,Entreprendre et innover,Marketing,Finance d'entreprise,Les enjeux de la transition écologique,...,Machine Learning et Applications,Ingénierie et systèmes Haute Tension,An Introduction to Meteorology and Oceanography,"Méthode des éléments finis, de la théorie à la...","Surfaces, friction, vibrations",Entreprendre et innover,Marketing,Les enjeux de la transition écologique,Communication interpersonnelle et pratiques pr...,Philosophie des sciences et techniques
3,al-mahdi.lahrache@ecl20.ec-lyon.fr,30.2,26.0,22.0,21.7,0.1,Théorie des probabilités et introduction aux p...,Analyse fonctionnelle : théorie et applications,Applications Web,Programmation des interfaces graphiques en C++,...,-,-,-,-,-,-,-,-,-,-
4,leandro.melo-de-freitas@ecl20.ec-lyon.fr,25.0,23.0,20.0,17.0,15.0,Finance d'entreprise,Marketing,Communication interpersonnelle et pratiques pr...,Savoir choisir un matériau,...,-,-,-,-,-,Finance d'entreprise,Marketing,Communication interpersonnelle et pratiques pr...,Savoir choisir un matériau,Des déchets et des hommes
5,gabriel.barbosa-paganini@ecl21.ec-lyon.fr,30.0,26.0,26.0,10.0,8.0,Propulseurs aéronautiques,Marketing,Finance d'entreprise,Conception d'un emballage responsable,...,Circuits et dispositifs en micro-ondes,Mathématiques appliquées à la biologie,"Méthode des éléments finis, de la théorie à la...",Théorie des probabilités et introduction aux p...,Analyse fonctionnelle : théorie et applications,Propulseurs aéronautiques,Marketing,Simulation multiphysique en conception mécanique,PLM - Maquette numérique,Mécanique des structures minces : Plaques et C...
6,filipe.penna@ecl21.ec-lyon.fr,41.0,35.0,11.0,10.0,3.0,Finance d'entreprise,Finance de marché,Applications Web,Programmation des interfaces graphiques en C++,...,Analyse fonctionnelle : théorie et applications,Space physics and solar-terrestrial coupling,"Ordre, chaos, fractales",Design optimal et mécanique des fluides numérique,Introduction aux vibrations aléatoires,Finance d'entreprise,Finance de marché,Ingénierie des Procédés Industriels,Systèmes autonomes de production (SAP),Les Métamatériaux Mécaniques
7,helena.palagi-gorck@ecl20.ec-lyon.fr,37.5,33.5,22.5,5.5,1.0,Finance d'entreprise,Marketing,Finance de marché,Eoliennes,...,Simulation multiphysique en conception mécanique,Space physics and solar-terrestrial coupling,"Surfaces, friction, vibrations",Théorie des probabilités et introduction aux p...,"Vivant, Information et Système",Finance d'entreprise,Marketing,Eoliennes,Des déchets et des hommes,Savoir choisir un matériau
8,matthieu.baboulene@ecl20.ec-lyon.fr,41.0,31.0,21.0,4.0,3.0,Finance de marché,Machine Learning et Applications,Algorithme et raisonnement,Finance d'entreprise,...,Optique et Photonique pour l'Ingénieur,Communication interpersonnelle et pratiques pr...,Ingénierie des Procédés Industriels,